<a href="https://colab.research.google.com/github/Ayush2803/GlobalHawks/blob/main/YOLO_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import cv2
import os
import datetime
import time
import numpy as np
import pandas as pd
import tensorflow as tf
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
from keras.losses import categorical_crossentropy
from keras.preprocessing.image import ImageDataGenerator
from keras.applications.vgg19 import VGG19, preprocess_input
from tensorflow.keras.optimizers import Adam
from keras.models import Model, Sequential
from keras.layers import Dense
from keras.layers import Flatten, Input
from keras.callbacks import ModelCheckpoint, EarlyStopping, TensorBoard
from math import sin, cos, radians

In [ ]:
IMAGE_SIZE = 224
CHANNELS = 3

In [ ]:
# Load dataset to memory:
images_folder = r'/content/drive/MyDrive/Dataset (1)'
labels = []
images = []

def resize(img, target):
    """ Resize an image """
    resized = cv2.resize(img, (target, target),
                         interpolation = cv2.INTER_AREA).astype(np.float32)
    return resized.reshape((-1))

for folder in os.listdir(images_folder):
    if folder in ('WithTarget','WithoutTarget'):
        for filename in os.listdir(os.path.join(images_folder,folder)):
            labels.append(folder)
            img = cv2.imread(os.path.join(images_folder,folder,filename))
            images.append(resize(img, IMAGE_SIZE))
            
images = np.array(images)
labels = pd.get_dummies(labels).to_numpy()

In [ ]:
# Data partitioning
x_train, x_test, y_train, y_test = train_test_split(
    images.reshape(-1,IMAGE_SIZE,IMAGE_SIZE,CHANNELS),
    labels, train_size=0.7, stratify=labels, random_state=42)

In [ ]:
def display_images(df):
    df = np.array(df)
    plt.figure(figsize=(10, 10))
    indices = np.random.choice(np.arange(len(df)),
                               9, replace=False)
    images = df[indices]
    for i in range(9):
        ax = plt.subplot(3, 3, i + 1)
        plt.imshow(cv2.cvtColor(images[i].reshape((224,224,3))
                                .astype('uint8'), cv2.COLOR_BGR2RGB))
        plt.axis("off")

In [ ]:
display_images(x_train[y_train[:,0]==1])

In [ ]:
display_images(x_train[y_train[:,1]==1])

In [ ]:
def add_noise(img):
    '''Add random noise to an image'''
    VARIABILITY = 50
    deviation = VARIABILITY*np.random.rand()
    noise = np.random.normal(0, deviation, img.shape)
    img += noise
    np.clip(img, 0., 255.)
    return img

# Generating augmentations
trdata = ImageDataGenerator(rescale=1./255, rotation_range=40,
                            width_shift_range=0.2, height_shift_range=0.2,
                            horizontal_flip=True, channel_shift_range=50,
                            preprocessing_function = add_noise)
traindata = trdata.flow(x_train, y_train,batch_size=32)

tsdata = ImageDataGenerator(rescale=1./255, rotation_range=40,
                            width_shift_range=0.2, height_shift_range=0.2,
                            horizontal_flip=True, channel_shift_range=50,
                            preprocessing_function = add_noise)
testdata = tsdata.flow(x_test, y_test,batch_size=32)

# Load VGG19 architecture as transfer learning model
vgg = VGG19(weights='imagenet', include_top=False,
            input_shape=(IMAGE_SIZE, IMAGE_SIZE, CHANNELS))
vgg.trainable = False
flatten = Flatten()(vgg.output)
d1 = Dense(2, activation="softmax")(flatten)
model = Model(inputs=vgg.input, outputs=d1)
model.summary()

In [ ]:
def exponential_decay(lr, s):
    def exponential_decay_fn(epoch):
        return lr * 0.1
    return exponential_decay_fn

exponential_decay_fn = exponential_decay(0.01, 5) # when i run it for 50 epochs

lr_scheduler = tf.keras.callbacks.LearningRateScheduler(exponential_decay_fn)

In [ ]:
model.compile(optimizer='Adam',
              loss=categorical_crossentropy, metrics=['accuracy'])
early = EarlyStopping(monitor='val_accuracy',
                      min_delta=0, verbose=1, patience=5, mode='auto')
hist = model.fit(traindata, steps_per_epoch=10, 
                 verbose=2, validation_data=testdata, validation_steps=5,
                 epochs=30, callbacks=lr_scheduler, shuffle=True)

Epoch 1/30
10/10 - 11s - loss: 1.0293 - accuracy: 0.7930 - val_loss: 0.7037 - val_accuracy: 0.8750 - lr: 0.0010 - 11s/epoch - 1s/step
Epoch 2/30
10/10 - 16s - loss: 0.4221 - accuracy: 0.8662 - val_loss: 0.4034 - val_accuracy: 0.9000 - lr: 0.0010 - 16s/epoch - 2s/step
Epoch 3/30
10/10 - 10s - loss: 0.2723 - accuracy: 0.9013 - val_loss: 0.4542 - val_accuracy: 0.7625 - lr: 0.0010 - 10s/epoch - 970ms/step
Epoch 4/30
10/10 - 10s - loss: 0.3310 - accuracy: 0.8822 - val_loss: 0.3416 - val_accuracy: 0.8500 - lr: 0.0010 - 10s/epoch - 980ms/step
Epoch 5/30
10/10 - 10s - loss: 0.3479 - accuracy: 0.8981 - val_loss: 0.3417 - val_accuracy: 0.8562 - lr: 0.0010 - 10s/epoch - 975ms/step
Epoch 6/30
10/10 - 10s - loss: 0.2724 - accuracy: 0.9076 - val_loss: 0.2948 - val_accuracy: 0.8687 - lr: 0.0010 - 10s/epoch - 973ms/step
Epoch 7/30
10/10 - 10s - loss: 0.4100 - accuracy: 0.8312 - val_loss: 0.4646 - val_accuracy: 0.7750 - lr: 0.0010 - 10s/epoch - 977ms/step
Epoch 8/30
10/10 - 12s - loss: 0.3806 - accurac

In [ ]:
acc = hist.history['accuracy']
val_acc = hist.history['val_accuracy']
loss = hist.history['loss']
val_loss = hist.history['val_loss']
epochs_range = range(1, len(hist.epoch) + 1)

plt.figure(figsize=(15,5))

plt.subplot(1, 2, 1)
plt.plot(epochs_range, acc, label='Train Set')
plt.plot(epochs_range, val_acc, label='Val Set')
plt.legend(loc="best")
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.title('Model Accuracy')

plt.subplot(1, 2, 2)
plt.plot(epochs_range, loss, label='Train Set')
plt.plot(epochs_range, val_loss, label='Val Set')
plt.legend(loc="best")
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.title('Model Loss')

plt.tight_layout()
plt.show()